# 4. Preparar Documentos
En este notebook se desarrolla el preprocesamiento necesario para la generación de embeddings a partir de los textos normativos, con el objetivo de indexar la información en una base Redis para su posterior consulta semántica y recuperación eficiente. El proceso abarca la preparación de los textos, el uso de modelos de lenguaje para obtener representaciones vectoriales (embeddings) y la estructuración de la información para su integración en el pipeline de gestión normativa.


In [239]:
# library
import pandas as pd
from typing import List, Union
import tiktoken
import redis as RedisClient
from redis.commands.search.field import TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType


from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.redis.base import Redis as RedisVectorStore

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from collections import Counter


import numpy as np
from redis.commands.search.query import Query
from IPython.display import display, HTML

import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from redis.commands.search.field import TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

from langchain_openai import OpenAIEmbeddings



In [ ]:
# -------------------------------
# CONSTANTS
# -------------------------------
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_DB = 0
REDIS_USERNAME = "default"
REDIS_PASSWORD = ""
REDIS_INDEX = "normativas_sii"
GPT_KEY = "sk-...."  # ← Reemplázalo por tu clave real

REDIS_URL = f"redis://{REDIS_USERNAME}:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}/{REDIS_DB}"
DATA_DIR ="../raw_data"
CSV_PATH = "03_classified_regulations.csv"
MODEL_NAME = "text-embedding-3-large"  # Modelo de OpenAI para embeddings
# Tokenizador para OpenAI embeddings
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
SAFE_TOKENS = 30000


## 4.1 Carga de Datos
En esta sección se realiza la carga de los datos normativos previamente preprocesados. Los datos contienen los textos completos y la metadata relevante asociada a cada normativa. Esta información será utilizada como insumo para la generación de los embeddings.

Se aplica un preprocesamiento básico a los textos normativos para optimizar la calidad de los embeddings generados. Esto puede incluir normalización de caracteres, eliminación de símbolos innecesarios y, en algunos casos, reducción de ruido textual. El objetivo es asegurar que la representación vectorial capture el contenido semántico relevante de cada normativa.

Utilizando un modelo de lenguaje preentrenado, se obtienen los embeddings para cada texto normativo. Estos vectores numéricos permiten comparar y recuperar información normativa de manera eficiente, habilitando funcionalidades avanzadas de búsqueda y clasificación semántica.




In [266]:
# -------------------------------
# FUNCTIONS
# -------------------------------
def contar_tokens(texto: str) -> int:
    """
    Counts the number of tokens in a given text using the default tokenizer.


    """
    return len(tokenizer.encode(texto))


def resumen_chunks_por_documento(documentos):
    """
    Imprime un resumen del número de chunks generados por cada documento.

    """
    conteo = Counter(doc.metadata.get("nombre", "Sin nombre") for doc in documentos)
    print(f"Total de chunks generados: {len(documentos)}")
    print("Resumen por documento (solo los que tienen más de un chunk):")
    for nombre, cantidad in conteo.items():
        if cantidad > 1:
            print(f"- {nombre}: {cantidad} chunk(s)")




def preparar_documentos(df: pd.DataFrame,
                        anios: Union[int, List[int]] = None,
                        chunk_size: int = None,
                        chunk_overlap: int = None) -> List[Document]:
    """
    Carga y prepara documentos desde un archivo CSV para procesamiento de NLP o indexación.
    """

    # Filtro por año
    if anios:
        if isinstance(anios, int):
            anios = [anios]
        # Convierte a int si tus años están como string
        df = df[df["anno"].isin(anios)]


    print(f"📄 Cargando {len(df)} documentos desde '{path_csv}' (filtrados por año: {anios})...")
    documentos = []
    splitter = None

    if chunk_size and chunk_overlap:
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ".", " "]
        )

    for _, row in df.iterrows():
        metadatos = {
            "nombre": row.get("nombre", ""),
            "descripcion": row.get("descripcion", ""),
            "fuente": row.get("fuente", ""),
            "url": row.get("url", ""),
            "tipo_documento": row.get("tipo_documento", ""),
            "relevancia": row.get("relevancia", ""),
            "explicacion": row.get("explicacion", ""),
            "anno": safe_int(row.get("anno", "")),
            "n_tokens": safe_int(contar_tokens(str(row["corpus"]).strip())),
            "exitos": safe_int(row.get("exitos", "")),
            "peso": safe_int(row.get("peso", "")),
          
        }
        cuerpo = str(row["corpus"]).strip()
        total_tokens = contar_tokens(cuerpo)
        print(f"🔍 Procesando documento: {row.get('nombre', '(sin nombre)')} - Tokens: {total_tokens}")
        if total_tokens <= SAFE_TOKENS:
            documentos.append(Document(page_content=cuerpo, metadata=metadatos))
        elif splitter:
            partes = splitter.split_text(cuerpo)
            for parte in partes:
                parte_limpia = parte.strip()
                if contar_tokens(parte_limpia) <= SAFE_TOKENS:
                    documentos.append(Document(page_content=parte_limpia, metadata=metadatos))
                else:
                    print(f"⚠️ Chunk excede el máximo seguro de tokens, se omitió parcialmente.")
        else:
            print(f"⚠️ Documento omitido por superar límite de tokens y no hay splitter definido.")

    return documentos



def buscar_knn_normativas(query: str, k: int = 5, min_score: float = 0.0):
    """
    Realiza una búsqueda KNN en Redis (con métrica COSINE) para encontrar normativas similares a una consulta.
    """
    
    # Obtener embedding
    embeddings = OpenAIEmbeddings(
        model=MODEL_NAME,  # Dimensión fija de 6144
        openai_api_key=GPT_KEY
    )
    query_vector = embeddings.embed_query(query)


    # Conexión Redis
    r = RedisClient.Redis(
        host=REDIS_HOST,
        port=int(REDIS_PORT),
        db=REDIS_DB,
        username=REDIS_USERNAME,
        password=REDIS_PASSWORD,
        decode_responses=True
    )

    # Búsqueda KNN
    base_query = f"*=>[KNN {k} @content_vector $vec as vector_score]"
    q = (
        Query(base_query)
        .return_fields("nombre","descripcion","relevancia", "explicacion", "url", "peso","exitos","vector_score")
        .sort_by("vector_score")  # Ordena por menor distancia
        .dialect(2)
    )

    results = r.ft("normativas_sii").search(
        q, query_params={"vec": np.array(query_vector, dtype=np.float32).tobytes()}
    )
    # Procesar resultados
    filas = []
    for doc in results.docs:
        distancia = float(doc.vector_score)
        similitud = 1 - distancia  # convertir a similitud 0-1
        if similitud >= min_score:
            nombre = getattr(doc, "nombre", "")
            url = getattr(doc, "url", "")
            nombre_link = f'<a href="{url}" target="_blank">{nombre}</a>' if url else nombre

            filas.append({
                "nombre": nombre_link,
                "descripcion": getattr(doc, "descripcion", ""),
                "peso": getattr(doc, "peso", ""),
                "exitos": getattr(doc, "exitos", ""),
                "relevancia": getattr(doc, "relevancia", ""),
                "explicacion": getattr(doc, "explicacion", ""),
                "vector_score": round(similitud, 4)  # ahora es similitud
            })

    # Mostrar en HTML
    df = pd.DataFrame(filas)
    display(HTML(df.to_html(escape=False, index=False)))

    return df


def safe_int(val):
    try:
        return int(val)
    except (ValueError, TypeError):
        return None

In [242]:



def delete_index(r):
    if not eliminar_anteriores:
        return
    print("🧹 Eliminando índice anterior...")
    try:
        r.ft(redis_index).dropindex(delete_documents=True)
        print(f"✅ Índice '{redis_index}' eliminado correctamente.")
    except Exception as e:
        print(f"ℹ️ No se pudo eliminar índice previo: {e}")

def indexar_en_redis_optimizado(documentos, redis_url, redis_index, gpt_key,
                               eliminar_anteriores=False,
                               modelo_embeddings="text-embedding-3-small",
                               batch_size=50, max_workers=5):
    """
    Indexa documentos en Redis optimizando:
    - Dimensiones (6144 para large, 1536 para small)
    - Procesamiento en paralelo
    - Manejo por lotes
    """

    r = RedisClient.Redis.from_url(redis_url)
    delete_index(r)
    
    # 🔹 Configurar dimensiones según el modelo
    DIM = 6144 if "large" in modelo_embeddings else 1536

    # 🔹 Inicializar embeddings
    embeddings = OpenAIEmbeddings(
        model=modelo_embeddings,
        openai_api_key=gpt_key,
    )


    # 🔹 Crear nuevo índice
    schema = [
        TextField("nombre"),
        TextField("descripcion"),
        TextField("fuente"),
        TextField("url"),
        TextField("tipo_documento"),
        TextField("relevancia"),
        TextField("explicacion"),
        NumericField("anno"),
        NumericField("n_tokens"),
        NumericField("peso"),
        NumericField("exitos"),
        VectorField("content_vector", "HNSW", {
            "TYPE": "FLOAT32",
            "DIM": DIM,
            "DISTANCE_METRIC": "COSINE"
        })
    ]

    try:
        r.ft(redis_index).create_index(
            fields=schema,
            definition=IndexDefinition(prefix=["doc:"], index_type=IndexType.HASH)
        )
        print(f"✅ Índice '{redis_index}' creado con COSINE y DIM={DIM}.")
    except Exception as e:
        print(f"ℹ️ Posible índice ya existente: {e}")

    errores = []
    total_indexados = 0

    # 🔹 Función para procesar lote
    def procesar_lote(lote):
        try:
            RedisVectorStore.from_documents(
                documents=lote,
                embedding=embeddings,
                redis_url=redis_url,
                index_name=redis_index
            )
            return len(lote), []
        except Exception as e:
            return 0, [{"error": str(e), "documentos": [d.metadata.get('nombre') for d in lote]}]

    # 🔹 Procesar en lotes y paralelo
    print(f"🚀 Iniciando indexación de {len(documentos)} documentos en lotes de {batch_size}...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for i in range(0, len(documentos), batch_size):
            lote = documentos[i:i+batch_size]
            futures.append(executor.submit(procesar_lote, lote))

        for future in tqdm(as_completed(futures), total=len(futures), desc="📦 Indexando"):
            count, errs = future.result()
            total_indexados += count
            errores.extend(errs)

    print(f"✅ Total indexados: {total_indexados}/{len(documentos)}")

    if errores:
        df_err = pd.DataFrame(errores)
        df_err.to_csv("errores_indexacion.csv", index=False, encoding="utf-8-sig")
        print("⚠️ Errores registrados en 'errores_indexacion.csv'")

    return total_indexados


## 4.2 Preprocesar Documentos
Se estructura la información para su integración en Redis, asignando claves únicas a cada normativa y asociando sus respectivos embeddings. Esta organización es fundamental para facilitar la recuperación rápida y precisa de normativas relevantes mediante consultas vectoriales.



In [ ]:
# Mostrar muestra de documentos cargados
path_csv=f"{DATA_DIR}/{CSV_PATH}"
df = pd.read_csv(path_csv)


documentos = preparar_documentos(
    df=df,
    anios=[2020,2021,2022,2023,2024],
    chunk_size=1000,
    chunk_overlap=200
)

In [254]:
print(f"📄 Se procesaran los documentos de los años [2020, 2021, 2022, 2023, 2024")
resumen_chunks_por_documento(documentos)

📄 Se procesaran los documentos de los años [2020, 2021, 2022, 2023, 2024
Total de chunks generados: 2910
Resumen por documento (solo los que tienen más de un chunk):
- Circular N° 31 del 19 de Mayo del 2021: 160 chunk(s)
- Circular N° 62 del 24 de Septiembre del 2020: 358 chunk(s)
- Circular N° 73 del 22 de Diciembre del 2020: 503 chunk(s)
- Circular N° 43 del 05 de Julio del 2021: 175 chunk(s)
- Circular N° 41 del 02 de Julio del 2021: 181 chunk(s)
- Circular N° 12 del 17 de Febrero del 2021: 274 chunk(s)
- Circular N° 53 del 10 de Agosto del 2020: 234 chunk(s)


# 4.3 Preparación para indexación en Redis
Se estructura la información para su integración en Redis, asignando claves únicas a cada normativa y asociando sus respectivos embeddings. Esta organización es fundamental para facilitar la recuperación rápida y precisa de normativas relevantes mediante consultas vectoriales.

Se realiza la conexión con la instancia de Redis y se procede a la carga de los embeddings generados junto con la metadata correspondiente. Este paso habilita la infraestructura para consultas semánticas, permitiendo la búsqueda y recomendación de normativas en función de la similitud entre textos.


In [ ]:
indexar_en_redis(documentos, REDIS_URL, REDIS_INDEX, GPT_KEY, eliminar_anteriores=True)

## 4.4 Validación del proceso
Se efectúan pruebas de validación sobre la carga y consulta de embeddings en Redis para asegurar la correcta integración del flujo. Se verifican tanto la recuperación de los vectores como la consistencia de la metadata asociada a cada normativa.


In [269]:
df = buscar_knn_normativas("¿Qué normativas boletas electrónicas?", k=2)



nombre,descripcion,peso,exitos,relevancia,explicacion,vector_score
Resolución Exenta SII N° 53 del 09 de Junio del 2022,"Modifica resolución Ex. SII N° 74 de fecha 02 de julio de 2020, eliminando obligación de enviar el resumen de ventas diarias en la forma y condiciones que indica.",3,2,Relevante,Contiene 'cumplimiento tributario' | Contiene 'boletas electrónicas',0.6456
Resolución Exenta SII N° 104 del 02 de Septiembre del 2020,"Modifica fecha de entrada en vigencia de la resolución Ex. SII N° 74 de fecha 02 de julio de 2020, que instruye procedimiento para emitir boletas electrónicas y boletas no afectas y exentas electrónicas de ventas y servicios.",2,1,No Relevante,Contiene 'boletas electrónicas',0.6381


In [268]:
df = buscar_knn_normativas("Encontrar medios de pago", k=2)


nombre,descripcion,peso,exitos,relevancia,explicacion,vector_score
Resolución Exenta SII N° 106 del 06 de Septiembre del 2023,"Aprueba Convenio Interadministrativo entre la Tesorería General de la República, el Banco del Estado de Chile y el Servicio de Impuestos Internos para disponibilizar canales de pago que faciliten la recaudación y el pago de tributos y otros ingresos públicos.",4,2,Relevante,Contiene 'pagos electrónicos' | Contiene 'cumplimiento tributario',0.3937
Resolución Exenta SII N° 91 del 21 de Agosto del 2020,Establece la obligación de informar las transacciones con tarjetas de pago según lo que se indica.,0,0,No Relevante,No se encontraron reglas de negocio,0.3777


## 4.5 Conclusiones

En esta etapa se desarrolló el preprocesamiento y la preparación de los textos normativos para la generación de embeddings vectoriales, con el propósito de habilitar la indexación y recuperación semántica eficiente en una base de datos Redis. El flujo de trabajo implementado incluyó la carga de datos preprocesados, la normalización y depuración adicional de los textos, y la obtención de representaciones vectoriales mediante modelos de lenguaje avanzados.

Posteriormente, los documentos y sus embeddings fueron organizados y estructurados para su integración en Redis, asignando claves únicas y asociando la metadata relevante. Se estableció un pipeline robusto para la indexación, que contempla la gestión de límites de tokens, la fragmentación inteligente de textos largos (chunking), y el control de errores en la carga masiva, asegurando la integridad y trazabilidad de los datos.

Este proceso permite habilitar funcionalidades de búsqueda y recuperación semántica avanzada, donde consultas en lenguaje natural pueden ser eficientemente mapeadas y comparadas contra el corpus normativo. La infraestructura construida, basada en Redis y modelos de embeddings de última generación, sienta las bases para el desarrollo de sistemas de recomendación, análisis automatizado y consulta inteligente de normativas legales, contribuyendo a la modernización y eficiencia en la gestión documental y normativa.

La metodología implementada garantiza la reproducibilidad, escalabilidad y calidad del sistema, posicionando la solución para aplicaciones futuras en análisis jurídico, cumplimiento normativo asistido por IA y automatización avanzada de flujos legales.
